# NER CRF - Atos de Contratos

## Configurando o ambiente

Se estiver utilizanod o VSCode na sua máquina local ou no servidor do projeto, o primeio passo é criar o ambiente virtual.

* Acesse a pasta onde vai colocar o seu projeto, no meu caso está em:

        /home/lucelia_vieira/Experimentos


* Crie o ambiente
    
        python3 -m venv 
    

* Para ativar o ambiente virtual localmente:
    
        source /home/lucelia_vieira/Experimentos/venv/bin/activate


A primeira vez que carregar o projeto no VScode execute:

#!virtualenv --python=python3.8 venv

Instale os pacotes abaixo direto no ambiente criado caso não queira ter que executar essa célula sempre que tiver que executar o projeto.

Para instalar direto no ambiente, o ambiente precisa estar ativo, conforme instrução para ativar o ambiente.

Se estiver utilizanodo o Collab, alguns desses pacotes já estarão disponíveis, outros não. Então, na dúvida, execute sempre...

In [157]:
#!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
#!pip install seqeval
#!pip install scikit-learn==0.24
#!pip install sklearn==0.0
#!pip install sklearn-crfsuite==0.3.6
#!pip install python-crfsuite==0.9.7
#!pip install tqdm
#!pip install pytorch-crf==0.7.2
#!pip install torch==1.8.1
#!pip install torch==1.11.0
#!pip install torch-summary
#!pip install nltk-3.7
#!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
#!pip install matplotlib-3.5.2
#!pip install tensorflow

Imports dos pacotes que serão utilizados no código

In [1]:
import pandas as pd
import nltk
import scipy.stats
import sklearn
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import matplotlib.pyplot as plt
%matplotlib inline
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     /home/lucelia_vieira/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Converte CSV para Padrão IOB

Ao carregar o csv pelo pandas especifique o data class das colunas. No nosso caso todas são String, então:

     dtype=str

In [2]:
data = pd.read_csv('/home/lucelia_vieira/Experimentos/ner/CSVs/DODFCorpus_contratos_licitacoes_v0.csv', dtype=str)

In [3]:
len(data)

30382

Para o iob_transformer fazer a correlação ato-entidades_correspondentes é necessário ter uma coluna id_ato, que junte o id do dodf e o id da relação específica.

Mas para que funcione corretamente as colunas id_dodf e id_rel devem ser do tipo String, conforme abaixo: 

 1   id_dodf       30382 non-null  object
 
 3   id_rel        30382 non-null  object

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30382 entries, 0 to 30381
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    30382 non-null  object
 1   id_dodf       30382 non-null  object
 2   tipo_rel      30382 non-null  object
 3   id_rel        30382 non-null  object
 4   anotador_rel  30382 non-null  object
 5   tipo_ent      30382 non-null  object
 6   id_ent        30382 non-null  object
 7   anotador_ent  30382 non-null  object
 8   offset        30382 non-null  object
 9   length        30382 non-null  object
 10  texto         30382 non-null  object
 11  id_ato        30382 non-null  object
dtypes: object(12)
memory usage: 2.8+ MB


In [5]:
data['id_ato'] = data['id_dodf'] + '-' + data['id_rel']
data

,Unnamed: 0,id_dodf,tipo_rel,id_rel,anotador_rel,tipo_ent,id_ent,anotador_ent,offset,length,texto,id_ato
0,0,7_2.8.2019,REL_ADITAMENTO_CONTRATO,R11,anotador_4624608,numero_termo_aditivo,140,anotador_4624608,372177,6,QUARTO,7_2.8.2019-R11
1,1,7_2.8.2019,REL_ADITAMENTO_CONTRATO,R11,anotador_4624608,numero_contrato,141,anotador_4624608,372213,8,002/2019,7_2.8.2019-R11
2,2,7_2.8.2019,REL_ADITAMENTO_CONTRATO,R11,anotador_4624608,numero_contrato,142,anotador_4624608,372231,6,033411,7_2.8.2019-R11
3,3,7_2.8.2019,REL_ADITAMENTO_CONTRATO,R11,anotador_4624608,processo_gdf,139,anotador_4624608,372260,19,00361-0 000853/2016,7_2.8.2019-R11
4,4,7_2.8.2019,REL_ADITAMENTO_CONTRATO,R11,anotador_4624608,orgao_contratante,143,anotador_4624608,372294,82,SECRETARIA DE ESTADO DE\nPROTECAO DA ORDEM URB...,7_2.8.2019-R11
...,...,...,...,...,...,...,...,...,...,...,...,...
30377,30394,631,REL_ADITAMENTO_CONTRATO,R157,anotador_4624608,orgao_contratante,2050,anotador_4624608,281629,38,Tribunal de Contas do Distrito Federal,631-R157
30378,30395,631,REL_ADITAMENTO_CONTRATO,R157,anotador_4624608,objeto_aditamento_contratual,2186,anotador_4624608,281779,477,"prestação dos serviços de locação de veículos,...",631-R157
30379,30396,631,REL_ADITAMENTO_CONTRATO,R157,anotador_4624608,processo_gdf,2052,anotador_4624608,282273,10,20065/2016,631-R157
30380,30397,631,REL_ADITAMENTO_CONTRATO,R157,anotador_4624608,data_escrito,2185,anotador_4624608,283587,10,23/12/2021,631-R157


Checando o tipo da coluna id_ato

In [6]:

type(data.at[30379, 'id_ato'])

str

Checando o formato do id_ato

In [7]:

data.at[30380, 'id_ato']

'631-R157'

Feito isso, podemos checar o número de labels por Ato.

In [8]:
result = data.groupby('tipo_rel')['id_ato'].nunique()
print(result.sort_values(ascending=False))


tipo_rel
REL_ADITAMENTO_CONTRATO     1260
REL_EXTRATO_CONTRATO        1223
REL_AVISO_LICITACAO          317
REL_ANUL_REVOG_LICITACAO      31
REL_SUSPENSAO_LICITACAO       26
REL_EXTRATO_CONVENIO          18
Name: id_ato, dtype: int64


Com os dados certinhos, podemos fazer uso do iob_transformer normalmente:

In [9]:
# return_df=False para retornar atos e labels, ou  return_df=True para retornar dataset
from iob_transformer import iob_transformer
iob = iob_transformer('id_ato','texto','tipo_ent', keep_punctuation=True, return_df=False)

É utilizados uns ilocs no código do transformer, então por via das dúvidas é bom dar um reset_index nos dataframes filtrados:

O primeiro filtro a ser aplicado é pelo tipo de ATO:

Ver com matheus....

In [10]:
data.tipo_rel.value_counts()

REL_EXTRATO_CONTRATO        16549
REL_ADITAMENTO_CONTRATO      9626
REL_AVISO_LICITACAO          3608
REL_ANUL_REVOG_LICITACAO      236
REL_EXTRATO_CONVENIO          184
REL_SUSPENSAO_LICITACAO       179
Name: tipo_rel, dtype: int64

In [11]:
tipo_ato = data.loc[data['tipo_rel'] == 'REL_AVISO_LICITACAO'].reset_index(drop=True)
tipo_ato


,Unnamed: 0,id_dodf,tipo_rel,id_rel,anotador_rel,tipo_ent,id_ent,anotador_ent,offset,length,texto,id_ato
0,52,7_2.8.2019,REL_AVISO_LICITACAO,R22,anotador_27234949,AVISO_LICITACAO,11,anotador_27234949,390033,841,AVISO DE ABERTURA\nPREGAO ELETRONICO POR SRP N...,7_2.8.2019-R22
1,53,7_2.8.2019,REL_AVISO_LICITACAO,R22,anotador_27234949,modalidade_licitacao,1,anotador_27234949,390051,17,PREGAO ELETRONICO,7_2.8.2019-R22
2,54,7_2.8.2019,REL_AVISO_LICITACAO,R22,anotador_27234949,numero_licitacao,2,anotador_27234949,390080,8,167/2019,7_2.8.2019-R22
3,55,7_2.8.2019,REL_AVISO_LICITACAO,R22,anotador_27234949,codigo_licitacao_sistema_compras,7,anotador_27234949,390096,6,926119,7_2.8.2019-R22
4,56,7_2.8.2019,REL_AVISO_LICITACAO,R22,anotador_27234949,tipo_objeto,4,anotador_27234949,390111,235,Aquisicao regular do material odontologico PAS...,7_2.8.2019-R22
...,...,...,...,...,...,...,...,...,...,...,...,...
3603,30164,631,REL_AVISO_LICITACAO,R171,anotador_27234949,data_abertura_licitacao,97,anotador_27234949,234888,22,08 de novembro de 2021,631-R171
3604,30165,631,REL_AVISO_LICITACAO,R171,anotador_27234949,valor_estimado_contratacao,98,anotador_27234949,234936,10,"197.252,50",631-R171
3605,30166,631,REL_AVISO_LICITACAO,R171,anotador_27234949,sistema_compras,100,anotador_27234949,235049,23,www.licitacoes-e.com.br,631-R171
3606,30167,631,REL_AVISO_LICITACAO,R171,anotador_27234949,objeto_licitacao,2277,anotador_27234949,234653,133,a aquisição de material de consumo - aquisição...,631-R171


In [12]:
tipo_ato.tipo_ent.value_counts()

numero_licitacao                    357
modalidade_licitacao                355
tipo_objeto                         325
AVISO_LICITACAO                     317
objeto_licitacao                    314
data_abertura_licitacao             310
nome_responsavel                    309
processo_gdf                        303
sistema_compras                     296
valor_estimado_contratacao          269
orgao_licitante                     234
codigo_licitacao_sistema_compras    219
Name: tipo_ent, dtype: int64

Alguns atos possuem entidades rotuladas incorretamente, ou que, a quantide de rótulos não seja representativo. Nesse caso, sugere-se remover essas entidades do dataset.

In [86]:
l = ['decisao_tcdf']

tipo_ato = tipo_ato.loc[~tipo_ato.tipo_ent.isin(l)]

Uma feito todos os filtros no dataset, podemos realizar o transfomer e converter para o formato IOB.

In [20]:
atos, labels = iob.transform(tipo_ato)

Para conferir se o foi realizado corretamente o iob ao dataset, imprima o retorno do transformer em formato de dataset:

In [14]:
iob_dataset = iob_transformer('id_ato', 'texto',
                      'tipo_ent', keep_punctuation=True, return_df=True)

In [15]:
dataset_iob = iob_dataset.transform(tipo_ato)
dataset_iob

,Sentence_idx,Word,Tag
0,-1,UNK,O
1,0,AVISO,O
2,0,DE,O
3,0,ABERTURA,O
4,0,PREGAO,B-modalidade_licitacao
...,...,...,...
56592,316,HILDA,I-nome_responsavel
56593,316,DO,I-nome_responsavel
56594,316,CARMO,I-nome_responsavel
56595,316,SILVA,I-nome_responsavel


Listando as tags após a conversão para IOB

In [16]:
tags = set()

for label in labels:
    for tag in label:
        tags.add(tag)
tags = list(tags)
num_tags = len(tags)

In [25]:
tags

['I-data_abertura_licitacao',
 'I-modalidade_licitacao',
 'I-valor_estimado_contratacao',
 'I-objeto_licitacao',
 'I-sistema_compras',
 'I-orgao_licitante',
 'B-data_abertura_licitacao',
 'B-numero_licitacao',
 'O',
 'I-numero_licitacao',
 'I-nome_responsavel',
 'B-orgao_licitante',
 'B-modalidade_licitacao',
 'I-codigo_licitacao_sistema_compras',
 'B-codigo_licitacao_sistema_compras',
 'I-processo_gdf',
 'B-processo_gdf',
 'B-nome_responsavel',
 'B-sistema_compras',
 'B-valor_estimado_contratacao',
 'B-objeto_licitacao',
 'B-tipo_objeto',
 'I-tipo_objeto']

In [26]:
x=atos
y=labels

In [27]:
len(y)

317

## Create dictionary feature for each word in each sequence in x

In [28]:
def get_features(sentence):
        """Create features for each word in act.
        Create a list of dict of words features to be used in the predictor module.
        Args:
            act (list): List of words in an act.
        Returns:
            A list with a dictionary of features for each of the words.
        """
        sent_features = []
        for i in range(len(sentence)):
            word_feat = {
                'word': sentence[i].lower(),
                'capital_letter': sentence[i][0].isupper(),
                'all_capital': sentence[i].isupper(),
                'isdigit': sentence[i].isdigit(),
                'word_before': sentence[i].lower() if i == 0 else sentence[i-1].lower(),
                'word_after:': sentence[i].lower() if i+1 >= len(sentence) else sentence[i+1].lower(),
                'BOS': i == 0,
                'EOS': i == len(sentence)-1
            }
            sent_features.append(word_feat)
        return sent_features
    
for i in range(len(x)):
    x[i] = get_features(x[i])

## Separate train and test splits (in order)

Aqui usamos trains_test_split do sklearn para separar os conjuntos de treino e teste de forma randômica de sistematizada.

In [46]:
from sklearn.model_selection import train_test_split
# 80% treino, 20% teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
print( 'x_train',len(x_train),'\n','x_test',len(x_test),'\n','total atos',len(x))


x_train 253 
 x_test 64 
 total atos 317


## Model CRF Trainning

In [47]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=10,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=False,
    all_possible_states=True
)

#crf.fit(x_train, y_train)

try:
    #crf.fit(x_train[:200], y_train[:200])
    crf.fit(x_train, y_train)
except AttributeError:
    pass


## Evaluation

In [48]:
classes = list(crf.classes_)
classes.remove('O')

y_pred = crf.predict(x_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=classes)

/home/lucelia_vieira/Experimentos/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1492: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


0.8098023904591443

In [65]:
crf.classes_

['O',
 'B-modalidade_licitacao',
 'I-modalidade_licitacao',
 'B-numero_licitacao',
 'B-processo_gdf',
 'B-tipo_objeto',
 'I-objeto_licitacao',
 'B-valor_estimado_contratacao',
 'I-valor_estimado_contratacao',
 'B-data_abertura_licitacao',
 'B-sistema_compras',
 'B-codigo_licitacao_sistema_compras',
 'B-nome_responsavel',
 'I-nome_responsavel',
 'B-orgao_licitante',
 'I-orgao_licitante',
 'B-objeto_licitacao',
 'I-processo_gdf',
 'I-tipo_objeto',
 'I-data_abertura_licitacao',
 'I-sistema_compras',
 'I-codigo_licitacao_sistema_compras',
 'I-numero_licitacao']

In [49]:
print(metrics.flat_classification_report(
    #y_test[:200], y_pred[:200], labels=classes, digits=3))
    y_test, y_pred, labels=classes, digits=3))



                                    precision    recall  f1-score   support

            B-modalidade_licitacao      0.932     0.809     0.866        68
            I-modalidade_licitacao      0.932     0.833     0.880        66
                B-numero_licitacao      0.941     0.696     0.800        69
                    B-processo_gdf      0.917     0.193     0.319        57
                     B-tipo_objeto      0.250     0.059     0.095        17
                I-objeto_licitacao      0.844     0.909     0.875      3040
      B-valor_estimado_contratacao      0.917     0.898     0.907        49
      I-valor_estimado_contratacao      0.000     0.000     0.000         0
         B-data_abertura_licitacao      0.857     0.522     0.649        69
                 B-sistema_compras      0.958     0.687     0.800        67
B-codigo_licitacao_sistema_compras      1.000     0.837     0.911        43
                B-nome_responsavel      0.970     0.500     0.660        64
           

/home/lucelia_vieira/Experimentos/venv/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=['B-modalidade_licitacao', 'I-modalidade_licitacao', 'B-numero_licitacao', 'B-processo_gdf', 'B-tipo_objeto', 'I-objeto_licitacao', 'B-valor_estimado_contratacao', 'I-valor_estimado_contratacao', 'B-data_abertura_licitacao', 'B-sistema_compras', 'B-codigo_licitacao_sistema_compras', 'B-nome_responsavel', 'I-nome_responsavel', 'B-orgao_licitante', 'I-orgao_licitante', 'B-objeto_licitacao', 'I-processo_gdf', 'I-tipo_objeto', 'I-data_abertura_licitacao', 'I-sistema_compras', 'I-codigo_licitacao_sistema_compras', 'I-numero_licitacao'] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/home/lucelia_vieira/Experimentos/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score ar

In [100]:
len(y_pred)

64

In [53]:
from sklearn.metrics import confusion_matrix
#confusion_matrix(y_test, y_pred, labels=classes)


import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix

cm = confusion_matrix(y_test, y_pred, labels=crf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=crf.classes_)
disp.plot() 


ValueError: You appear to be using a legacy multi-label data representation. Sequence of sequences are no longer supported; use a binary array or sparse matrix instead - the MultiLabelBinarizer transformer can convert to this format.

## Hyperparameter Optimization

In [ ]:
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True,
    all_possible_states=True
)
params_space = {
    'c1': scipy.stats.expon(scale=15.0),
    'c2': scipy.stats.expon(scale=1.0),
}


# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=classes)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(x_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


/home/lucelia_vieira/Experimentos/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1492: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/home/lucelia_vieira/Experimentos/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1492: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/home/lucelia_vieira/Experimentos/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1492: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/home/lucelia_vieira/Experimentos/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1492: UndefinedMetricWarn

RandomizedSearchCV(cv=3,
                   estimator=CRF(algorithm='lbfgs', all_possible_states=True,
                                 all_possible_transitions=True,
                                 max_iterations=100),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f94c402f1f0>,
                                        'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f94c22621c0>},
                   scoring=make_scorer(flat_f1_sco...eto_contrato', 'I-numero_convenio', 'B-cnpj_entidade_convenente', 'B-vigencia_contrato', 'I-vigencia_contrato', 'B-valor_convenio', 'B-unidade_orcamentaria', 'B-programa_trabalho', 'B-natureza_despesa', 'B-fonte_recurso', 'B-cnpj_orgao_concedente', 'B-data_assinatura_contrato', 'I-data_assinatura_convenio', 'I-unidade_orcamentaria', 'B-nota_empenho']),
                   verbose=1)

In [ ]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 1.6608103444597717, 'c2': 0.1444627787858007}
best CV score: 0.22201714397578617
model size: 0.02M


## Check best estimator on our test data

In [ ]:
sorted_classes = sorted(
    classes,
    key=lambda name: (name[1:], name[0])
)

classes = list(crf.classes_)
classes.remove('O')

y_pred = crf.predict(x_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=classes)

In [ ]:
crf = rs.best_estimator_
y_pred = crf.predict(x_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_classes, digits=3
))

                            precision    recall  f1-score   support

B-cnpj_entidade_convenente      0.000     0.000     0.000         3
   B-cnpj_orgao_concedente      0.000     0.000     0.000         0
B-data_assinatura_contrato      0.000     0.000     0.000         0
B-data_assinatura_convenio      0.000     0.000     0.000         7
I-data_assinatura_convenio      0.000     0.000     0.000         4
     B-entidade_convenente      0.000     0.000     0.000         4
     I-entidade_convenente      0.000     0.000     0.000        13
           B-fonte_recurso      0.000     0.000     0.000         0
        B-natureza_despesa      0.000     0.000     0.000         0
            B-nota_empenho      0.000     0.000     0.000         0
         B-numero_convenio      0.000     0.000     0.000         1
         I-numero_convenio      0.000     0.000     0.000         0
         B-objeto_contrato      0.000     0.000     0.000         0
         I-objeto_contrato      0.000     0.000

/home/lucelia_vieira/Experimentos/venv/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=['B-cnpj_entidade_convenente', 'B-cnpj_orgao_concedente', 'B-data_assinatura_contrato', 'B-data_assinatura_convenio', 'I-data_assinatura_convenio', 'B-entidade_convenente', 'I-entidade_convenente', 'B-fonte_recurso', 'B-natureza_despesa', 'B-nota_empenho', 'B-numero_convenio', 'I-numero_convenio', 'B-objeto_contrato', 'I-objeto_contrato', 'B-objeto_convenio', 'I-objeto_convenio', 'B-orgao_concedente', 'I-orgao_concedente', 'B-processo_gdf', 'B-programa_trabalho', 'B-unidade_orcamentaria', 'I-unidade_orcamentaria', 'B-valor_convenio', 'B-vigencia_contrato', 'I-vigencia_contrato', 'B-vigencia_convenio', 'I-vigencia_convenio'] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/home/lucelia_vieira/Experimentos/venv/lib/python3.8/site-pac

## Let’s check what classifier learned

In [ ]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
I-objeto_contrato -> I-objeto_contrato 4.978239
I-objeto_convenio -> I-objeto_convenio 4.943337
I-vigencia_convenio -> I-vigencia_convenio 4.905194
O      -> O       4.837518
I-vigencia_contrato -> I-vigencia_contrato 4.671743
I-orgao_concedente -> I-orgao_concedente 4.191762
I-entidade_convenente -> I-entidade_convenente 4.000036
I-unidade_orcamentaria -> I-unidade_orcamentaria 3.662939
B-entidade_convenente -> I-entidade_convenente 3.465615
B-vigencia_convenio -> I-vigencia_convenio 3.175660
B-orgao_concedente -> I-orgao_concedente 3.086817
I-data_assinatura_convenio -> I-data_assinatura_convenio 3.005101
B-objeto_convenio -> I-objeto_convenio 2.701662
B-objeto_contrato -> I-objeto_contrato 1.677831
O      -> B-numero_convenio 1.311854
B-cnpj_entidade_convenente -> O       1.182061
B-processo_gdf -> O       1.175920
B-natureza_despesa -> O       1.037393
O      -> B-entidade_convenente 0.943200
B-numero_convenio -> O       0.941699

Top unlikely transitions:
B

In [ ]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
4.051562 O        word:.
3.669421 O        word::
3.526741 B-valor_convenio word_before:$
2.949884 B-valor_convenio word_after::(
2.723425 B-data_assinatura_convenio word:20/07/2021
2.495876 B-nota_empenho all_capital
2.365068 O        EOS
1.988892 B-fonte_recurso isdigit
1.905428 B-numero_convenio word_before:nº
1.903125 B-entidade_convenente word_before:a
1.862926 B-numero_convenio word_after::processo
1.855043 O        word:,
1.774928 B-data_assinatura_convenio word_before::
1.705451 B-fonte_recurso word:100
1.656255 B-numero_convenio word_before:no
1.643583 I-entidade_convenente capital_letter
1.596077 B-nota_empenho word_after::,
1.549822 O        word:-
1.548344 O        word_after::secretaria
1.506840 B-orgao_concedente word_after::de
1.500928 O        word:nº
1.420184 B-objeto_convenio word_before::
1.394982 O        word:;
1.379059 I-orgao_concedente capital_letter
1.368043 B-unidade_orcamentaria word_before::
1.353539 B-vigencia_convenio word_before::
1.280260 B

## Saving the best model

In [ ]:
import joblib

joblib.dump(crf, 'crf_model.pkl')

['crf_model.pkl']

In [ ]:
model = joblib.load('crf_model.pkl')

In [ ]:
model

CRF(algorithm='lbfgs', all_possible_states=True, all_possible_transitions=True,
    c1=1.6608103444597717, c2=0.1444627787858007, max_iterations=100)

In [ ]:
model.classes_

['O',
 'B-numero_convenio',
 'B-orgao_concedente',
 'I-orgao_concedente',
 'B-entidade_convenente',
 'I-entidade_convenente',
 'B-objeto_convenio',
 'I-objeto_convenio',
 'B-vigencia_convenio',
 'I-vigencia_convenio',
 'B-data_assinatura_convenio',
 'B-processo_gdf',
 'B-objeto_contrato',
 'I-objeto_contrato',
 'I-numero_convenio',
 'B-cnpj_entidade_convenente',
 'B-vigencia_contrato',
 'I-vigencia_contrato',
 'B-valor_convenio',
 'B-unidade_orcamentaria',
 'B-programa_trabalho',
 'B-natureza_despesa',
 'B-fonte_recurso',
 'B-cnpj_orgao_concedente',
 'B-data_assinatura_contrato',
 'I-data_assinatura_convenio',
 'I-unidade_orcamentaria',
 'B-nota_empenho']